<a href="https://colab.research.google.com/github/ort-eila/csiseminar/blob/main/projects/step2_mibi_DCIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import os
root_dir = "/content/gdrive/MyDrive/Seminar/project/immune_310_project"
dataset_path=os.path.join(root_dir,"dataset","NPY_41_channels","labels")
dataset_path

'/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels'

In [3]:
progressors_npy_dir = os.path.join(dataset_path,"progressor")
print("progressors_npy_dir is {}".format(progressors_npy_dir))
non_progressors_npy_dir = os.path.join(dataset_path,"nonprogressor")
print("non_progressors_npy_dir is {}".format(non_progressors_npy_dir))


progressors_npy_dir is /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor
non_progressors_npy_dir is /content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/nonprogressor


In [4]:
progressors_npy_files = os.listdir(progressors_npy_dir)
non_progressors_npy_files = os.listdir(non_progressors_npy_dir)
npy_n_channels_all = [os.path.join(progressors_npy_dir,x) for x in progressors_npy_files]
npy_n_channels_all = npy_n_channels_all + [os.path.join(non_progressors_npy_dir,x) for x in non_progressors_npy_files]
print(npy_n_channels_all[0:5])
print(npy_n_channels_all[-5:])


['/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/6103.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/2306.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/5310.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/6102.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/progressor/3105.npz']
['/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/nonprogressor/5306.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/nonprogressor/5308.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/labels/nonprogressor/2320.npz', '/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/NPY_41_channels/l

In [5]:
# Step 2. create 224x224 images for the model and save them in a seperate foldet
# with a folder name = dataset name

In [6]:
import numpy as np
from PIL import Image
import re
import os

def create_rgb_images_from_channels(channel_data, output_shape, output_path, filename_prefix):
    height, width = channel_data[0].shape
    channels_per_rgb = 3


    rgb_values = []
    # for channel_index in range(0, len(channel_data)-1, channels_per_rgb):
    #   print("channel_index is {}".format(channel_index))
    for col in range(0, width):
      for row in range(0, height):
        for channel_index in range(0, len(channel_data)-1, channels_per_rgb):
          # print("channel_index is {}".format(channel_index))
          # print("row is {}".format(row))
          # print("col is {}".format(col))
          R,G,B = 0,0,0 # prepare the padding
          R = channel_data[channel_index][row, col]
          if (channel_index+1 <= (len(channel_data)-1)):
            G = channel_data[channel_index+1][row, col]
          if (channel_index+2 <= (len(channel_data)-1)):
            B = channel_data[channel_index+2][row, col]
          rgb_val = (R,G,B)
          if rgb_val == (0,0,0):
            # print("rgb_val is {}. Continue".format(rgb_val))
            continue
          # else:
          #   print("rgb_val {} will be added".format(rgb_val))
          # print()
          rgb_values.append(rgb_val)

    # print("rgb_values is {}".format(rgb_values))
    rgb_values_counter = 0
    # Create the rgb_image using the accumulated rgb_values
    num_of_images = len(rgb_values) // (output_shape[0]*output_shape[1]) + 1
    # create at list one image
    for img in range(num_of_images):
      rgb_image = np.zeros((output_shape[0], output_shape[1], 3), dtype=np.uint8)
      for img_i in range(output_shape[0]):
        for img_j in range(output_shape[1]):
          rgb_val = (0,0,0)
          if rgb_values_counter < (len(rgb_values) - 1):
            rgb_val = rgb_values[rgb_values_counter]
            rgb_values_counter+=1
            rgb_image[img_i,img_j,] = rgb_val
            # print("rgb_image[img_i,img_j] is {}".format(rgb_image[img_i,img_j]))
      pil_image = Image.fromarray(rgb_image)
      file_name = f'{filename_prefix}_rgb_image_img{img}_row{row}_col{col}.png'
      full_file_name = f'{output_path}/{file_name}'
      print("Save file name {}".format(full_file_name))
      pil_image.save(full_file_name)

In [7]:
# # execution example
# # Combine channel data
# channel_data = [channel1, channel2, channel3, channel4, channel5, channel6, channel7, channel8, channel9, channel10]
# # Define output path and filename prefix
# output_path = os.path.join(root_dir,'output_folder')
# print("output_path is {}".format(output_path))

# os.makedirs(output_path,exist_ok=True)
# filename_prefix = 'output_prefix'
# print("filename_prefix is {}".format(filename_prefix))
# output_shape = (224, 224)


for npz_file_path in npy_n_channels_all:
    # default params: rgb_height=224, rgb_width=224
    # Replace the pattern using regular expressions
    print("npz_file_path is {}".format(npz_file_path))
    output_shape = (224, 224)
    num_channels = 42
    npz_file_fld = os.path.dirname(npz_file_path)
    print("npz_file_fld is {}".format(npz_file_fld))
    # output_path = npz_file_fld.replace("NPY_41_channels","JPG_RGB_224_224")
    output_path = re.sub(r'NPY_.*_channels', 'JPG_RGB_{}_{}_NO_ZEROS'.format(output_shape[0], output_shape[1]), npz_file_fld)
    print("output_path is {}".format(output_path))
    os.makedirs(output_path, exist_ok=True)
    patient = os.path.basename(npz_file_path).split(".npz")[0]
    # Load the NPZ file
    # Load the NPZ file
    import numpy as np

    # Load the NPZ file
    loaded_data = np.load(npz_file_path)

    # Check the keys present in the loaded NPZ file
    print("Keys in the NPZ file:", loaded_data.files)

    # Now you have the extracted channel_data as a list of numpy arrays
    channel_data = loaded_data['channel_data']

    # Call the function to create and save RGB images
    # DEF: reate_rgb_images_from_channels(channel_data, output_shape, output_path, filename_prefix)
    create_rgb_images_from_channels(channel_data, output_shape, output_path, patient)


Streaming output truncated to the last 5000 lines.
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val (3, 0, 0) will be added
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val (3, 0, 0) will be added
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val (8, 0, 0) will be added
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val (8, 0, 0) will be added
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 0, 0). Continue
rgb_val is (0, 

KeyboardInterrupt: ignored

In [ ]:
# output_path = "/content/gdrive/MyDrive/Seminar/project/immune_310_project/dataset/JPG_RGB_224_224/labels/progressor"

In [ ]:

# import os
# os.path.dirname(output_path)

In [ ]:
# root_folder = os.path.dirname(output_path)
# root_folder

In [ ]:
# os.path.join(os.path.dirname(root_folder),"train_validation_test")

In [ ]:
# create the train / validation and test folder for the created images:

import os
import shutil
import random

# Define the root folder containing the labels and images
root_folder = os.path.dirname(output_path)
print("root_folder is {}".format(root_folder))
output_folder = os.path.join(os.path.dirname(root_folder),"train_validation_test") # Define the output folder where train, validation, and test sets will be created
print("output_folder is {}".format(output_folder))

# Define label names
label_names = os.listdir(root_folder)

# Define train, validation, and test ratios
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# Create train, validation, and test folders
for split in ['train', 'validation', 'test']:
    split_folder = os.path.join(output_folder, split)
    os.makedirs(split_folder, exist_ok=True)
    for label in label_names:
        os.makedirs(os.path.join(split_folder, label), exist_ok=True)

# Iterate through each label
for label in label_names:
    label_folder = os.path.join(root_folder, label)
    image_files = [f for f in os.listdir(label_folder) if f.endswith('.png')]

    random.shuffle(image_files)

    num_samples = len(image_files)
    num_train = int(num_samples * train_ratio)
    num_validation = int(num_samples * validation_ratio)

    train_images = image_files[:num_train]
    validation_images = image_files[num_train:num_train + num_validation]
    test_images = image_files[num_train + num_validation:]

    for image in train_images:
        src = os.path.join(label_folder, image)
        dest = os.path.join(output_folder, 'train', label, image)
        shutil.copy(src, dest)

    for image in validation_images:
        src = os.path.join(label_folder, image)
        dest = os.path.join(output_folder, 'validation', label, image)
        shutil.copy(src, dest)

    for image in test_images:
        src = os.path.join(label_folder, image)
        dest = os.path.join(output_folder, 'test', label, image)
        shutil.copy(src, dest)

print("Data splitting and copying complete.")



In [ ]:
# type(test_dataset)

In [ ]:
train_dataset

In [ ]:
ex = train_dataset['image'][0]
ex

In [ ]:
# image = ex['image']
# image

In [ ]:
labels = train_dataset['labels'][0]
labels

In [ ]:
label_string = class_label.int2str(labels)
label_string

In [ ]:
# labels.ste(train_dataset['labels'][0])
model_name_or_path = 'google/vit-base-patch16-224-in21k'
# Preprocessing function for images


In [ ]:
# # Define a function to preprocess your custom dataset
# def preprocess_custom_dataset(example):
#     image = Image.open(example['image_file_path'])
#     inputs = feature_extractor(images=image, return_tensors="pt")
#     return {
#         'pixel_values': inputs['pixel_values'],
#         'label': example['label']  # Replace with the actual way you load labels
#     }

In [ ]:
# from transformers import ViTFeatureExtractor, ViTForImageClassification
# from transformers import Trainer, TrainingArguments
# from datasets import load_dataset
# import torch
# from PIL import Image

# def preprocess_image(example):
#     # Load the image using PIL
#     image = Image.open(example['image_file_path'])
#     # Use the feature extractor to preprocess the image
#     inputs = feature_extractor(images=image, return_tensors="pt")
#     return inputs

# # Load the feature extractor
# feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# preprocessed_train_and_validation_dataset = train_and_validation_dataset.map(preprocess_image)
# preprocessed_test_dataset = test_dataset.map(preprocess_image)
# # Load the ViT model
# model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')
# # device = 'cuda' if torch.cuda.is_available() else 'cpu'
# # model.to(device)  # Move the model to the desired device

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Set up training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     evaluation_strategy="steps",
#     eval_steps=500,
#     logging_dir='./logs',
#     save_total_limit=3,
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=preprocessed_train_and_validation_dataset,
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# eval_results = trainer.evaluate(preprocessed_test_dataset)
# print(eval_results)




In [ ]:
# from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
# from PIL import Image
# import torch

# # Define a function to preprocess and extract patches from your custom dataset
# def preprocess_custom_dataset(example):
#     image = Image.open(example['image_file_path'])

#     # Resize the image to the desired model input size (224x224)
#     image = image.resize((224, 224))
#     #!!TODO: change to patches if the resize works for the model, or think of something else
#     # Use the feature extractor to preprocess the image and extract patches
#     inputs = feature_extractor(images=image, return_tensors="pt")

#     return {
#         'pixel_values': inputs['pixel_values'],
#         'labels': example['labels']  # Replace with the actual way you load labels
#     }

# # Load the feature extractor
# feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# # Your existing custom Dataset object
# # custom_dataset =train_and_validation_dataset

# # Preprocess the custom dataset and extract patches
# preprocessed_custom_dataset = train_and_validation_dataset.map(preprocess_custom_dataset)

# # Load the ViT model
# model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k')

# # Set up training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     evaluation_strategy="steps",
#     eval_steps=500,
#     logging_dir='./logs',
#     save_total_limit=3,
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=preprocessed_custom_dataset,
# )

# # Train the model
# trainer.train()


In [ ]:
# preprocessed_custom_dataset['pixel_values']

In [ ]:
# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=preprocessed_train_and_validation_dataset,
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# eval_results = trainer.evaluate(preprocessed_test_dataset)
# print(eval_results)


In [ ]:
# # Set up training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     evaluation_strategy="epoch",
#     logging_dir='./logs',
#     save_total_limit=3,
# )

# # Set up training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
#     per_device_train_batch_size=8,
#     evaluation_strategy="steps",
#     eval_steps=500,
#     logging_dir='./logs',
#     save_total_limit=3,
# )

# # Initialize the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=preprocessed_train_and_validation_dataset,
# )

# # Train the model
# trainer.train()

# # Evaluate the model
# eval_results = trainer.evaluate(preprocessed_test_dataset)
# print(eval_results)

